In [1]:
import matplotlib as mpl
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow.compat.v1 as tf
from sklearn.preprocessing import LabelEncoder
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
import time
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import roc_auc_score
import scipy.stats as st
from sympy.solvers import solve
from sympy import Symbol
from scipy.optimize import fsolve
from sklearn.metrics import fbeta_score,f1_score,balanced_accuracy_score,recall_score,precision_score
from sklearn.neural_network import MLPClassifier
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from cvxopt import matrix
from cvxopt import solvers
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Activation, GlobalAveragePooling2D,Conv2D, MaxPooling2D
from sklearn.ensemble import AdaBoostClassifier # AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier # KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier 
from tensorflow.keras.models import Model, Sequential
import math

/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/lvjingzhe/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym

In [2]:
yeast=pd.read_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/DRM/data/yeast.csv',)

In [265]:
yeast

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,Class
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,MIT
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,MIT
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,MIT
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,NUC
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,MIT
...,...,...,...,...,...,...,...,...,...
1479,0.82,0.63,0.47,0.18,0.5,0.0,0.50,0.22,POX
1480,0.35,0.29,0.46,0.14,0.5,0.0,0.54,0.66,NUC
1481,0.41,0.47,0.50,0.14,0.5,0.0,0.53,0.37,NUC
1482,0.51,0.45,0.62,0.25,0.5,0.0,0.59,0.22,MIT


In [3]:
yeast.iloc[:,-1].value_counts()

 CYT    463
 NUC    429
 MIT    244
 ME3    163
 ME2     51
 ME1     44
 EXC     35
 VAC     30
 POX     20
 ERL      5
Name:  Class, dtype: int64

In [4]:
L=[]
for k in yeast[' Class'].values:
    if k ==' ME2':
        L.append(1)
    else:L.append(0)
yeast[' Class']=L

In [5]:
yeast[' Class'].value_counts()

0    1433
1      51
Name:  Class, dtype: int64

In [6]:
E=np.array(yeast)
np.random.seed(4123)
np.random.shuffle(E)
X_sample=E[:,:-1]/np.max(E[:,:-1],axis=0)
y_sample=E[:,-1]
X_sample.shape

(1484, 8)

In [166]:
# X_sample=np.array(ecoli.iloc[:,:-1]/np.max(ecoli.iloc[:,:-1],axis=0))
# y_sample=np.array(ecoli['Class'])
# X_sample.shape

In [7]:
X_sample_XX=np.c_[X_sample,np.ones(X_sample.shape[0])]

In [8]:
X_sample_ord=np.concatenate((X_sample[y_sample==0],X_sample[y_sample==1]))
y_sample_ord=np.concatenate((y_sample[y_sample==0],y_sample[y_sample==1]))

In [9]:
X_sample_ord.shape

(1484, 8)

In [10]:
x_tr, x_te, y_tr, y_te = train_test_split(X_sample,y_sample,test_size = 0.2,
                                                  shuffle = True,
                                                  random_state = 4123)

In [11]:
X_train=np.c_[x_tr,np.ones(x_tr.shape[0])]
X_test=np.c_[x_te,np.ones(x_te.shape[0])]

In [40]:
x_te.shape,x_tr.shape

((297, 8), (1187, 8))

In [13]:
y_sample.shape[0]/sum(y_sample)

29.098039215686274

In [14]:
def split(x,y):
    k=int(y.shape[0]/sum(y))-1
    n=y[y==1].shape[0]
    m=int((x.shape[0]-n)/k)
    indice=[i*m for i in range(1,k)]
    indice.append(x.shape[0]-n)
    x_c=np.concatenate((x[y==0],x[y==1]))
    y_c=np.concatenate((np.array([0]*(x.shape[0]-n)),np.array([1]*n)))
    return np.array(np.split(x_c,indice)),np.array(np.split(y_c,indice))


n=y_tr[y_tr==1].shape[0]
XX=np.concatenate((X_train[y_tr==0],X_train[y_tr==1]))
YY=np.concatenate((np.array([0]*(X_train.shape[0]-n)),np.array([1]*n)))
m=int((X_train.shape[0]-n)/68)
indice=[k*m for k in range(1,68)]
indice.append(X_train.shape[0]-n)

pre_XX=np.array(np.split(XX,indice))
pre_YY=np.array(np.split(YY,indice))

In [99]:
# cv_= RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=0)
# cv=RepeatedStratifiedKFold(n_splits=3, n_repeats=1, random_state=42)
cv= RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=0)
cv_=RepeatedStratifiedKFold(n_splits=10, n_repeats=1, random_state=0)

In [16]:
import time
start_i=time.time()
model_in=LogisticRegression()
model_in.fit(x_tr,y_tr)
# model_in.fit(X_resampled_smote,y_resampled_smote)
end_i=time.time()
y_train_proba=model_in.predict_proba(x_tr)
y_train_label=model_in.predict(x_tr)
y_test_proba=model_in.predict_proba(x_te)

y_test_label=model_in.predict(x_te)

from sklearn.metrics import classification_report
print(classification_report(y_te,y_test_label))
print("Running time:%.2f s"%(end_i-start_i))
from sklearn.metrics import accuracy_score

print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_test_label)))
print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_test_label)))
print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_test_label,beta=2)))
print('Accuracy on training set：{:.2%}'.format(accuracy_score(y_tr,y_train_label)))
print('Accuracy on testing set：{:.2%}'.format(accuracy_score(y_te,y_test_label)))

              precision    recall  f1-score   support

         0.0       0.95      1.00      0.98       283
         1.0       0.00      0.00      0.00        14

    accuracy                           0.95       297
   macro avg       0.48      0.50      0.49       297
weighted avg       0.91      0.95      0.93       297

Running time:0.01 s
Balanced-Accuracy on testing set：50.00%
Recall on testing set：0.00%
F-measure on testing set：0.00%
Accuracy on training set：96.88%
Accuracy on testing set：95.29%


## DRM

In [163]:
class Poly_close_solution:
    def __init__(self,kernel='self._poly',penalty = None,gamma=1,degree = 2,coef0=0,alpha=1,beta=1):
        self.W = None
        self.penalty = penalty
        self.d = degree
        self.b = coef0
        self.alpha=alpha
        self.beta=beta
        self.g=gamma
        self.kernel=kernel
        
    def _poly(self,X_1,X_2):
        return (self.g*(X_1.dot(X_2.T))+self.b)**self.d
    def _rbf(self,x,y):
        return np.exp(-self.g*np.sum((x[...,None,:]-y)**2,axis=2))

    def poly_B_matrix(self,X,Y):
        n=Y[Y==1].shape[0]
        X_sample_split=np.array(np.split(X,np.array([X.shape[0]-n])))
        B=np.zeros((X.shape[0],X.shape[0]))
        I=0
        for  m in X_sample_split:
            I+=m.shape[0]
            B[I-m.shape[0]:I,I-m.shape[0]:I]=eval(self.kernel+'(m,m)')/m.shape[0]
        return B
    
    def K_x(self,X,x_t):
        return eval(self.kernel+'(X,x_t)')
    
    def QplusBeta(self,X,Y):
        s=time.time()
        K=eval(self.kernel+'(X,X)')
        H_p=np.diag(np.diagonal(K))
        B_p=self.poly_B_matrix(X,Y)
        Qplus_beta=K+self.alpha*(H_p-B_p)+np.diag([self.beta]*X.shape[0])
        e=time.time()
        return K,Qplus_beta
    def fit(self,invQ,X,x_t):
        
        
        W=invQ.dot(self.K_x(X,x_t))##求逆耗时
        
        return W.reshape(-1)
    def delta_phi(self,K,invQ,X,Y,x_t,j):
        
        w=self.fit(invQ,X,x_t)
        w_noty=w

        w_y=np.zeros((X.shape[0]))
        indice=np.argwhere(Y==j).reshape(-1)
        w_noty[indice]=0
        w_y[indice]=w[indice]
        delta=np.dot(w_y,K.dot(w_y))+np.dot(w_noty,K.dot(w_noty))-2*np.dot(w_y,self.K_x(X,x_t))
        return delta
    def predict(self,K,invQ,X,Y,x_t):
        delta=[]
        for k in np.unique(Y):
            delta.append(self.delta_phi(K,invQ,X,Y,x_t,k))
        return np.argmin(delta)

In [164]:
from numpy.linalg import inv,det

In [165]:
from sklearn.metrics import fbeta_score
from numpy.linalg import inv,det
metrics_names=['G-mean','Balanced_acc','F1_score','Recall','Precision']
def DRM_FiveCV(X,y,model=Poly_close_solution(
                                )):
    
    G_m,bacc,f2,rec,pre=list(),list(),list(),list(),list()
    for train_index,test_index in cv_.split(X,y):
                
                K,Q_beta=model.QplusBeta(X[train_index],y[train_index])
                invQ=inv(Q_beta)
                y_test_label=list()
                for i in range(y[test_index].shape[0]):
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                    y_test_label.append(model.predict(K,invQ,X[train_index],y[train_index],X[test_index][i]))
#                 y_train_proba=model.predict_proba(X[train_index])
#                 y_train_label=model.predict(X[train_index])
                
#                 print("running time for each CV:%.2f s"%(t2-t1))
                
#     print(classification_report(y,y_pre))
                rec.append(recall_score(y[test_index],y_test_label))
                pre.append(precision_score(y[test_index],y_test_label))
                G_m.append(math.sqrt(recall_score(y[test_index],y_test_label)*recall_score(y[test_index],y_test_label,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[test_index],y_test_label))
                f2.append(fbeta_score(y[test_index],y_test_label,beta=2))
 
    return G_m,bacc,f2,rec,pre


In [45]:
degree=np.array([2,3,4,5,8,10])##tr:te=8:2
gamma=np.array([0.01,0.1,1,10])
parameter=np.array([0.01,0.1,1,10,100])
X_tr_ord=np.concatenate((x_tr[y_tr==0],x_tr[y_tr==1]))
y_tr_ord=np.concatenate((y_tr[y_tr==0],y_tr[y_tr==1]))
all_results_Ldrm = []
import time
import math
# for k in mean_lr:
start=time.time()

for k in gamma:
    for p in gamma:
        for d in degree:
            Lo_score=np.mean(DRM_FiveCV(X_tr_ord,y_tr_ord,model=Poly_close_solution(alpha=k,beta=p,degree=d
                                )),axis=1)

            metric_res = {'alpha':k,'beta':p,'degree':d}

            for name, value in zip(metrics_names, Lo_score):
        #             print(name, ': ', value)
                    metric_res[name] = value


            all_results_Ldrm.append(metric_res)
end=time.time()
print("DRM_L_fivefold-cv time:%.2f"%(end-start))

DRM_L_fivefold-cv time:218.55


In [46]:
pd.DataFrame(all_results_Ldrm).to_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/code/0514CV_select_DRML_keelyeast_4.csv')


In [47]:
degree=np.array([2,3,4,5,8,10])
gamma=np.array([0.001,0.01,0.1,1,10])
parameter=np.array([0.01,0.1,1,10,100])
def fivetrials_DRM(X,y):
    
    
    G_m,bacc,fbeta,rec,pre,T=list(),list(),list(),list(),list(),list()
    
    for train_index,test_index in cv.split(X, y):
#         all_results_Ldrm = []
#         for k in parameter:
#             for p in parameter:
#                 for d in degree:
#                     Lo_score=np.mean(DRM_FiveCV(X[train_index],y[train_index],model=Poly_close_solution(alpha=k,beta=p,degree=d
#                                         )),axis=1)

#                     metric_res = {'alpha':k,'beta':p,'degree':d}

#                     for name, value in zip(metrics_names, Lo_score):
#                 #             print(name, ': ', value)
#                             metric_res[name] = value


#                     all_results_Ldrm.append(metric_res)
#         eva=pd.DataFrame(all_results_Ldrm)
#         bias=eva[eva.iloc[:,-5]==(eva.iloc[:,-5]).max()]
#         al,be,de=np.array(bias[['alpha','beta','degree']])[0]
        start=time.time()
        P_model=Poly_close_solution(alpha=1,beta=0.1,degree=2)
        K,Q_beta=P_model.QplusBeta(X[train_index],y[train_index])
        invQ=inv(Q_beta)
        prdict_y=[]
        for i in range(y[test_index].shape[0]):
            prdict_y.append(P_model.predict(K,invQ,X[train_index],y[train_index],X[test_index][i]))
        end=time.time()
        
        print(classification_report(y[test_index],prdict_y))
        rec.append(recall_score(y[test_index],prdict_y))
        G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
        pre.append(precision_score(y[test_index],prdict_y))
    #     auc.append(roc_auc_score(y[test_index],prob_y[:,-1]))
        bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
        fbeta.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(y_sample.shape[0]/sum(y_sample)-1))))
        T.append(end-start)
#     pipeline = Pipeline(steps=steps)
   
#     print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_L)))
#     print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_L)))
#     print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_L,beta=3.66)))
#     print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_d-start_d)))



    return G_m,bacc,fbeta,rec,pre,T

In [48]:
result_DRM = fivetrials_DRM(X_sample_ord,y_sample_ord)

G_mean=result_DRM[0]
Bacc=result_DRM[1]
# summarize performance
recall=result_DRM[3]
Fbeta=result_DRM[2]
T=result_DRM[-1]
acc_s=np.mean(np.array(result_DRM)[:4,:],axis=0)
	# summarize and store
print('>%s: Average G-mean:%.3f(%.3f) ' % ('DRM:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('DRM:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('DRM:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('DRM:',np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % ('DRM:',np.mean(T),np.std(T)))
print('>%s: Average accuracy_score: %.3f(%.3f)' % ('DRM:',np.mean(acc_s),np.std(acc_s)))
print('>%s: Average Score: %.3f(%.3f)' % ('DRM:',np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

              precision    recall  f1-score   support

         0.0       0.99      0.92      0.95       287
         1.0       0.20      0.60      0.30        10

    accuracy                           0.91       297
   macro avg       0.59      0.76      0.62       297
weighted avg       0.96      0.91      0.93       297

              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96       287
         1.0       0.23      0.50      0.31        10

    accuracy                           0.93       297
   macro avg       0.60      0.72      0.64       297
weighted avg       0.96      0.93      0.94       297

              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96       287
         1.0       0.31      0.90      0.46        10

    accuracy                           0.93       297
   macro avg       0.65      0.92      0.71       297
weighted avg       0.97      0.93      0.95       297

              preci

# SOTA methods

In [17]:
from sklearn import tree
def get_models():
	models, names = list(), list()
	# LR
	models.append(LogisticRegression())
	names.append('LR')
	# SVM
	models.append(SVC(probability=True))
	names.append('SVM')
	# Bagging
	models.append(BaggingClassifier(n_estimators=1000))
	names.append('BAG')
	models.append(AdaBoostClassifier(random_state=0))
	names.append('Adaboost')
	# RF
	models.append(
	tree.DecisionTreeClassifier(max_leaf_nodes=6,
                               random_state=0))
	names.append('CART')
	models.append(RandomForestClassifier(criterion='entropy',n_estimators=6))
	names.append('RF')
	# GBM
	models.append(MLPClassifier(random_state=0, max_iter=200,hidden_layer_sizes=(50,)))
    
	names.append('MLP')
# 	models.append(solution_XX())
# 	names.append('max-mean loss')
	return models, names

In [18]:
def evaluate_model(X,y,model):
#     steps = [('p', PowerTransformer()), ('m',model)]
    
#     pipeline = Pipeline(steps=steps)
    pipeline = model
    import time
    start=time.time()
    pipeline.fit(X,y)
    end=time.time()
    prdict_y=pipeline.predict(x_te)
    prob_y=pipeline.predict_proba(x_te)
    rec=recall_score(y_te,prdict_y)
    G_m=np.sqrt(recall_score(y_te,prdict_y)*recall_score(y_te,prdict_y,pos_label=0))
    pre=precision_score(y_te,prdict_y)
#     auc.append(roc_auc_score(y[test_index],prob_y[:,-1]))
    bacc=balanced_accuracy_score(y_te,prdict_y)
    fbeta=fbeta_score(y_te,prdict_y,beta=4)
#     print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_L)))
#     print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_L)))
#     print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_L,beta=3.66)))
#     print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_d-start_d)))



    return G_m,bacc,fbeta,rec,pre,end-start

In [19]:
import time
def fivetrials(X,y,model,sample='none'):
    pipeline = model
    
    G_m,bacc,fbeta,rec,pre,T=list(),list(),list(),list(),list(),list()
    if sample!='none':
#     steps = [('p', PowerTransformer()), ('m',model)]
        for train_index,test_index in cv.split(X, y):
        
            start=time.time()
            resample_x,resample_y=sample.fit_resample(X[train_index],y[train_index])
            
            pipeline.fit(resample_x,resample_y)
            end=time.time()
            prdict_y=pipeline.predict(X[test_index])
            prob_y=pipeline.predict_proba(X[test_index])
            rec.append(recall_score(y[test_index],prdict_y))
            G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
            pre.append(precision_score(y[test_index],prdict_y))
        #     auc.append(roc_auc_score(y[test_index],prob_y[:,-1]))
            bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
            fbeta.append(fbeta_score(y[test_index],prdict_y,beta=2))
            T.append(end-start)
#     pipeline = Pipeline(steps=steps)
    else:
        for train_index,test_index in cv.split(X, y):
    
            start=time.time()
            pipeline.fit(X[train_index],y[train_index])
            end=time.time()
            prdict_y=pipeline.predict(X[test_index])
            prob_y=pipeline.predict_proba(X[test_index])
            rec.append(recall_score(y[test_index],prdict_y))
            G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
            pre.append(precision_score(y[test_index],prdict_y))
        #     auc.append(roc_auc_score(y[test_index],prob_y[:,-1]))
            bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
            fbeta.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(y_sample.shape[0]/sum(y_sample)-1))))
            T.append(end-start)
#     print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_L)))
#     print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_L)))
#     print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_L,beta=3.66)))
#     print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_d-start_d)))



    return G_m,bacc,fbeta,rec,pre,T
# print('>%s: Mean Balanced_Acc: %.3f (%.3f)' % (names[-1],mean(Bacc),std(Bacc)))

In [20]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
X_resampled,y_resampled=RandomUnderSampler().fit_resample(x_tr,y_tr)
X_resampled_smote,y_resampled_smote  = SMOTE().fit_resample(x_tr, y_tr)

In [166]:
models, names = get_models()
# from sklearn.neural_network import MLPClassifier
# evaluate each model
for i in range(len(models)):
	# evaluate the model and store results
	result = fivetrials(X_sample,y_sample,models[i])

	G_mean=result[0]
	Bacc=result[1]
# summarize performance
	recall=result[3]
	Fbeta=result[2]
	T=result[-1]
	acc_s=np.mean(np.array(result)[:4,:],axis=0)
	# summarize and store
	print('>%s: Average G-mean:%.3f(%.3f) ' % (names[i],np.mean(G_mean),np.std(G_mean)))
	print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % (names[i],np.mean(Bacc),np.std(Bacc)))
	print('>%s: Average Fbeta: %.3f(%.3f)' % (names[i],np.mean(Fbeta),np.std(Fbeta)))
	print('>%s: Average Recall: %.3f(%.3f)' % (names[i],np.mean(recall),np.std(recall)))    
	print('>%s: Average Training time: %.3f(%.3f)' % (names[i],np.mean(T),np.std(T)))
	print('>%s: Average accuracy_score: %.3f(%.3f)' % (names[i],np.mean(acc_s),np.std(acc_s)))
	print('>%s: Average Score: %.3f(%.3f)' % (names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

>LR: Average G-mean:0.000(0.000) 
>LR: Average Balanced_Acc: 0.500(0.000) 
>LR: Average Fbeta: 0.000(0.000)
>LR: Average Recall: 0.000(0.000)
>LR: Average Training time: 0.015(0.007)
>LR: Average accuracy_score: 0.125(0.000)
>LR: Average Score: 0.134(0.000)
>SVM: Average G-mean:0.000(0.000) 
>SVM: Average Balanced_Acc: 0.500(0.000) 
>SVM: Average Fbeta: 0.000(0.000)
>SVM: Average Recall: 0.000(0.000)
>SVM: Average Training time: 0.055(0.010)
>SVM: Average accuracy_score: 0.125(0.000)
>SVM: Average Score: 0.133(0.000)
>BAG: Average G-mean:0.521(0.039) 
>BAG: Average Balanced_Acc: 0.633(0.020) 
>BAG: Average Fbeta: 0.287(0.040)
>BAG: Average Recall: 0.275(0.039)
>BAG: Average Training time: 2.060(0.058)
>BAG: Average accuracy_score: 0.429(0.034)
>BAG: Average Score: 0.428(0.034)
>Adaboost: Average G-mean:0.464(0.127) 
>Adaboost: Average Balanced_Acc: 0.610(0.059) 
>Adaboost: Average Fbeta: 0.240(0.119)
>Adaboost: Average Recall: 0.235(0.118)
>Adaboost: Average Training time: 0.081(0.006)

In [21]:
models, names = get_models()
# from sklearn.neural_network import MLPClassifier
# evaluate each model
for i in range(len(models)):
	# evaluate the model and store results
	result = fivetrials(X_sample,y_sample,models[i],sample=SMOTE(random_state=42))

	G_mean=result[0]
	Bacc=result[1]
# summarize performance
	recall=result[3]
	Fbeta=result[2]
	T=result[-1]
	acc_s=np.mean(np.array(result)[:4,:],axis=0)
	# summarize and store
	print('>%s: Average G-mean:%.3f(%.3f) ' % ('SMOTE'+names[i],np.mean(G_mean),np.std(G_mean)))
	print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('SMOTE'+names[i],np.mean(Bacc),np.std(Bacc)))
	print('>%s: Average Fbeta: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(Fbeta),np.std(Fbeta)))
	print('>%s: Average Recall: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(recall),np.std(recall)))    
	print('>%s: Average Training time: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(T),np.std(T)))
	print('>%s: Average accuracy_score: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(acc_s),np.std(acc_s)))
	print('>%s: Average Score: %.3f(%.3f)' % ('SMOTE'+names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          

>SMOTELR: Average G-mean:0.793(0.055) 
>SMOTELR: Average Balanced_Acc: 0.799(0.051) 
>SMOTELR: Average Fbeta: 0.433(0.048)
>SMOTELR: Average Recall: 0.727(0.110)
>SMOTELR: Average Training time: 0.017(0.001)
>SMOTELR: Average accuracy_score: 0.688(0.065)
>SMOTELR: Average Score: 0.691(0.064)
>SMOTESVM: Average G-mean:0.789(0.066) 
>SMOTESVM: Average Balanced_Acc: 0.798(0.058) 
>SMOTESVM: Average Fbeta: 0.447(0.057)
>SMOTESVM: Average Recall: 0.709(0.129)
>SMOTESVM: Average Training time: 0.449(0.017)
>SMOTESVM: Average accuracy_score: 0.686(0.076)
>SMOTESVM: Average Score: 0.686(0.075)
>SMOTEBAG: Average G-mean:0.676(0.111) 
>SMOTEBAG: Average Balanced_Acc: 0.725(0.083) 
>SMOTEBAG: Average Fbeta: 0.428(0.126)
>SMOTEBAG: Average Recall: 0.491(0.168)
>SMOTEBAG: Average Training time: 5.498(0.135)
>SMOTEBAG: Average accuracy_score: 0.580(0.122)
>SMOTEBAG: Average Score: 0.576(0.121)
>SMOTEAdaboost: Average G-mean:0.720(0.100) 
>SMOTEAdaboost: Average Balanced_Acc: 0.749(0.078) 
>SMOTEAdab

In [167]:
models, names = get_models()
# from sklearn.neural_network import MLPClassifier
# evaluate each model
for i in range(len(models)):
	# evaluate the model and store results
	result = fivetrials(X_sample,y_sample,models[i],sample=RandomUnderSampler(random_state=42))

	G_mean=result[0]
	Bacc=result[1]
# summarize performance
	recall=result[3]
	Fbeta=result[2]
	T=result[-1]
	acc_s=np.mean(np.array(result)[:4,:],axis=0)
	# summarize and store
	print('>%s: Average G-mean:%.3f(%.3f) ' % ('RUS'+names[i],np.mean(G_mean),np.std(G_mean)))
	print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('RUS'+names[i],np.mean(Bacc),np.std(Bacc)))
	print('>%s: Average Fbeta: %.3f(%.3f)' % ('RUS'+names[i],np.mean(Fbeta),np.std(Fbeta)))
	print('>%s: Average Recall: %.3f(%.3f)' % ('RUS'+names[i],np.mean(recall),np.std(recall)))    
	print('>%s: Average Training time: %.3f(%.3f)' % ('RUS'+names[i],np.mean(T),np.std(T)))
	print('>%s: Average accuracy_score: %.3f(%.3f)' % ('RUS'+names[i],np.mean(acc_s),np.std(acc_s)))
	print('>%s: Average Score: %.3f(%.3f)' % ('RUS'+names[i],np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          


>RUSLR: Average G-mean:0.810(0.065) 
>RUSLR: Average Balanced_Acc: 0.814(0.063) 
>RUSLR: Average Fbeta: 0.426(0.057)
>RUSLR: Average Recall: 0.784(0.133)
>RUSLR: Average Training time: 0.007(0.002)
>RUSLR: Average accuracy_score: 0.708(0.079)
>RUSLR: Average Score: 0.711(0.078)
>RUSSVM: Average G-mean:0.830(0.067) 
>RUSSVM: Average Balanced_Acc: 0.833(0.065) 
>RUSSVM: Average Fbeta: 0.445(0.062)
>RUSSVM: Average Recall: 0.824(0.132)
>RUSSVM: Average Training time: 0.004(0.001)
>RUSSVM: Average accuracy_score: 0.733(0.081)
>RUSSVM: Average Score: 0.736(0.081)
>RUSBAG: Average G-mean:0.811(0.034) 
>RUSBAG: Average Balanced_Acc: 0.813(0.034) 
>RUSBAG: Average Fbeta: 0.416(0.056)
>RUSBAG: Average Recall: 0.804(0.064)
>RUSBAG: Average Training time: 1.283(0.020)
>RUSBAG: Average accuracy_score: 0.711(0.040)
>RUSBAG: Average Score: 0.708(0.039)
>RUSAdaboost: Average G-mean:0.759(0.073) 
>RUSAdaboost: Average Balanced_Acc: 0.761(0.072) 
>RUSAdaboost: Average Fbeta: 0.344(0.079)
>RUSAdaboost: 

In [261]:
y_sample.shape[0]/10

75.2

# Max-mean loss method

In [64]:
pre_YY.shape

(30,)

In [168]:
class solution_XX:
    
    
    def __init__(self,penalty = None,Lambda = 0.03,a = 0.5,epochs = 200):
        self.W = None
        self.penalty = penalty
        self.Lambda = Lambda
        self.a = a
        self.epochs =epochs
        self.sigmoid = lambda x:1/(1 + np.exp(-x))
        
#     err=(y_tr-s.sigmoid(np.dot(X_train,w)))**2
#     err[index]=2.6*err[index]

    def f_XX(self,X,Y):
        if self.penalty=='l1':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(np.abs(self.W)) for x,y in zip(X,Y)])
        elif self.penalty=='l2':f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 )+self.Lambda*np.sum(self.W**2) for x,y in zip(X,Y)])#pre_Xtrain,pre_Ytrain
        else:f=np.array([np.mean((self.sigmoid(np.dot(x,self.W))-y)**2 ) for x,y in zip(X,Y)])

        return f         
    def Gf_XX(self,X,Y):#To compute the Derivative matrix, the shape of which is N*2
        if self.penalty=='l1':d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+self.Lambda*np.sign(self.W )for x,y in zip(X,Y)])
        elif self.penalty=='l2':
            d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W))))+2*self.Lambda*self.W for x,y in zip(X,Y)])
#     return d.reshape(20,12289)
        else:d=np.array([x.T.dot(0.02*(self.sigmoid(np.dot(x,self.W))-y)*self.sigmoid(np.dot(x,self.W))*(1-self.sigmoid(np.dot(x,self.W)))) for x,y in zip(X,Y)])
        return d
    def direction_XX(self,X,Y):
        gra=self.Gf_XX(X,Y)
        p=matrix(gra.dot(gra.T),tc='d')
        q=matrix(-self.f_XX(X,Y),tc='d')
        G=matrix(np.diag(np.array([-1]*(Y.shape[0]))),tc='d')#N=20
        h=matrix(np.array([[0]]*(Y.shape[0])),tc='d')
        A=matrix([[1.0]]*(Y.shape[0]))
        b=matrix([1.0])
        solvers.options['show_progress'] = False
        sol = solvers.qp(p,q,G,h,A,b)
        t=np.array(sol['x'])
        d= -(gra.T.dot(t))
        return d.reshape((X_train.shape[-1],))

    def fit(self,X,Y):
        
        call=[]
        pre=[]
        loss=[]
        testloss=[]
        np.random.seed(1324)
        self.W=np.random.random((X_train.shape[-1],))*2-1
#         self.W=w_2
        n=y_te[y_te==1.].shape[0]
        for k in range(200):
    #     while np.linalg.norm(d)//10**(-8) >= 10:
            d=self.direction_XX(X,Y)
#             print(np.linalg.norm(d))
            if np.linalg.norm(d)//10**(-7) < 25:
                break
            sigma=0.8
            f_1=np.max(self.f_XX(X,Y))
            w=self.W
            self.W=d*sigma+w
            while np.max(self.f_XX(X,Y))>np.max(f_1):
                sigma=sigma*0.8
                self.W=d*sigma+w
            self.W=d*sigma+w

            
    #         output=output.reshape(4000,)
    #         pt=max(output[output==1.].shape[0],1)
    #         m=0
    #         for j in range(4000):
    #             if output[j]==test_label[j]==1:
    #                 m+=1


    # # b1=np.random.random((1,10))*2-1


            
    #         call.append(m/n)
    #         pre.append(m/pt)
    #         loss.append(max(f_X(w)))
    #         testloss.append(ff_X(w))
        return self.W

In [70]:
X_train.shape

(1279, 12)

In [169]:
def fivetrials_XX(X,y,model=solution_XX()):
    pipeline = model
    
    G_m,bacc,fbeta,rec,pre,T=list(),list(),list(),list(),list(),list()
    
#     steps = [('p', PowerTransformer()), ('m',model)]
    for train_index,test_index in cv.split(X, y):
        
        start=time.time()
#         n=y[train_index][y[train_index]==1].shape[0]
#         XX=np.concatenate((X[train_index][y[train_index]==0],X[train_index][y[train_index]==1]))
#         YY=np.concatenate((np.array([0]*(X[train_index].shape[0]-n)),np.array([1]*n)))
#         m=int((X[train_index].shape[0]-n)/3)
#         indice=[k*m for k in range(1,3)]
#         indice.append(X[train_index].shape[0]-n)

#         pre_X=np.array(np.split(XX,indice))
#         pre_Y=np.array(np.split(YY,indice))
        pre_X,pre_Y=split(X[train_index],y[train_index])
#         print(pre_Y.shape)

        w_x=pipeline.fit(pre_X,pre_Y)
        end=time.time()
        prdict_y=pipeline.sigmoid(X[test_index].dot(w_x))
        prdict_y[prdict_y>=0.5]=1
        prdict_y[prdict_y<0.5]=0

        print(classification_report(y[test_index],prdict_y))
        
        rec.append(recall_score(y[test_index],prdict_y))
        G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
        pre.append(precision_score(y[test_index],prdict_y))
    #     auc.append(roc_auc_score(y[test_index],prob_y[:,-1]))
        bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
        fbeta.append(fbeta_score(y[test_index],prdict_y,beta=2))
        T.append(end-start)
#     pipeline = Pipeline(steps=steps)
   
#     print('Balanced-Accuracy on testing set：{:.2%}'.format(balanced_accuracy_score(y_te,y_L)))
#     print('Recall on testing set：{:.2%}'.format(recall_score(y_te,y_L)))
#     print('F-measure on testing set：{:.2%}'.format(fbeta_score(y_te,y_L,beta=3.66)))
#     print('Score :{:.3}'.format(np.mean([recall,bacc,fbeta])*0.99+0.01/(end_d-start_d)))



    return G_m,bacc,fbeta,rec,pre,T

In [71]:
X_sample_XX.shape

(1599, 12)

In [170]:
result_XX = fivetrials_XX(X_sample_XX,y_sample)

G_mean=result_XX[0]
Bacc=result_XX[1]
# summarize performance
recall=result_XX[3]
Fbeta=result_XX[2]
T=result_XX[-1]
acc_s=np.mean(np.array(result_XX)[:4,:],axis=0)
	# summarize and store
print('>%s: Average G-mean:%.3f(%.3f) ' % ('max_mean loss:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('max_mean loss:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('max_mean loss:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('max_mean loss:',np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % ('max_mean loss:',np.mean(T),np.std(T)))
print('>%s: Average accuracy_score: %.3f(%.3f)' % ('max_mean loss:',np.mean(acc_s),np.std(acc_s)))
print('>%s: Average Score: %.3f(%.3f)' % ('max_mean loss:',np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))
          


              precision    recall  f1-score   support

         0.0       0.98      0.90      0.94       287
         1.0       0.17      0.60      0.26        10

    accuracy                           0.89       297
   macro avg       0.58      0.75      0.60       297
weighted avg       0.96      0.89      0.92       297

              precision    recall  f1-score   support

         0.0       0.98      0.86      0.92       287
         1.0       0.13      0.60      0.21        10

    accuracy                           0.85       297
   macro avg       0.56      0.73      0.57       297
weighted avg       0.96      0.85      0.89       297

              precision    recall  f1-score   support

         0.0       1.00      0.85      0.92       287
         1.0       0.17      0.90      0.29        10

    accuracy                           0.85       297
   macro avg       0.58      0.87      0.60       297
weighted avg       0.97      0.85      0.89       297

              preci

# Mean-uncertain method

In [26]:
import testjx
def sigmoid_(x,u):
    return 1.0/(1.0+np.exp(-x-u))

In [108]:
fsolve(lambda x:equa_m(x,400,1),0.5 )

array([0.1490997])

In [943]:
start=time.time()
mean_sm_=np.array([fsolve(lambda x:equa_m(x,n),0.5 ) for n in range(10,600,5)])

end=time.time()
print(end-start)

429.49512219429016


In [27]:

def equa_m(x,n,r):
    pre_in=(np.dot(x_tr,model_in.coef_.T)+model_in.intercept_).reshape(x_tr.shape[0],)
    
    ini_err=y_tr-sigmoid_(pre_in,x)##ini_err is the predicted err of training set, based on LR

    for k in range(ini_err.shape[0]):
        if y_tr[k]==1:
            ini_err[k]=r*ini_err[k]

#     tm=testjx.testjx.meanuncertainty(ini_err.tolist(),n)
    return testjx.testjx.meanuncertainty(ini_err.tolist(),n)[1]
def equa_sm(x,n):
#     ini_err=y_tr-sigmoid_(pre_in,x)
    
    ini_err=y_tr-sigmoid_(pre_in,x)##ini_err is the predicted err of training set, based on LR

#     for k in ini_err.index:
#         if y_tr[k]==1:
#             ini_err[k]=1.5*ini_err[k]

#     tm=testjx.testjx.meanuncertainty(ini_err.tolist(),n)
    return testjx.testjx.meanuncertainty(ini_err.tolist(),n)[1]

In [28]:

def equa_fivem(x,n,r,Mod,x_train,y_train):
    pre_in=(np.dot(x_train,Mod.coef_.T)+Mod.intercept_).reshape(x_train.shape[0],)
    
    ini_err=y_train-sigmoid_(pre_in,x)##ini_err is the predicted err of training set, based on LR

    for k in range(ini_err.shape[0]):
        if y_train[k]==1:
            ini_err[k]=r*ini_err[k]

    
    return testjx.testjx.meanuncertainty(ini_err.tolist(),n)[1]


In [31]:
metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def evaluate_model_lr(X,y,m,model=LogisticRegression()):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
    for train_index,test_index in cv_.split(X,y):
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                model.fit(X[train_index],y[train_index])
                prob_y=sigmoid_((np.dot(X[test_index],model.coef_.T)+model.intercept_),m)
                
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=4))
 
    return bacc,f2,rec,pre

In [155]:
var_lr=np.array(pd.read_csv('/Users/lvjingzhe/Desktop/璇/modified_althogram/code/0512_lrvar_mean_yeast4.csv'))


In [321]:
# mean_xh=np.array([fsolve(lambda x:equa_xh(x,n,0.5*61),0.5 ) for n in range(50,420,5)])

In [60]:
def select(arr,n,k):##arr为1-D
    per=[]
    A=abs(arr-k)

    for j in np.sort(A)[:n]:
        per.append(list(A).index(j))
    return arr[per]

In [151]:
metrics_names=['Balanced_acc','F2_score','Recall','Precision']
def Fivetrails_mean(k,X,y,model=LogisticRegression()):
    
    mean=mean_lr.reshape(-1)
    G_m,bacc,f2,rec,pre,T=list(),list(),list(),list(),list(),list()
    for train_index,test_index in cv.split(X,y):
                s_1=time.time()
#                 print(sum(y[train_index]))
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                model.fit(X[train_index],y[train_index])
                t_1=time.time()
#                 mean_u=np.array([fsolve(lambda x:equa_fivem(x,n,0.5*(k+1),model,X[train_index],y[train_index]),0.5 ) for n in range(50,int(y[train_index].shape[0]*0.8),5)])
                all_results = []
                c=0
                
                
                for i in mean:
                    result=np.mean(evaluate_model_lr(X[train_index],y[train_index],i),axis=1)
                    metric_res = {'window':10+5*c,'upper_mean': i}
                    c+=1
                    for name, value in zip(metrics_names, result):
                #             print(name, ': ', value)
                            metric_res[name] = value


                    all_results.append(metric_res)
                eva=pd.DataFrame(all_results)
#                 eva.to_csv('{:}: datapre_diabetesCV_which_mu_to_set_inLRBIAS.csv'.format(k))
                bias=eva[eva.iloc[:,-4]==eva.iloc[:,-4].max()]
                mean=select(mean_lr.reshape(-1),5,bias['upper_mean'].mean())
                print("Optimal upper_mean selected by CV: %.3f "%(bias['upper_mean'].mean()))
                prob_y=sigmoid_((np.dot(X[test_index],model.coef_.T)+model.intercept_).reshape(X[test_index].shape[0],),bias['upper_mean'].mean()
                             )
                
                
                
                prdict_y=np.round(prob_y)
                print(classification_report(y[test_index],prdict_y))
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(k))))
                T.append(t_1-s_1)
    return G_m,bacc,f2,rec,pre,T


In [152]:
result = Fivetrails_mean(y_sample.shape[0]/sum(y_sample)-1,X_sample,y_sample)
# 
G_mean=result[0]
Bacc=result[1]
# summarize performance
recall=result[3]
Fbeta=result[2]
T=result[-1]
acc_s=np.mean(np.array(result)[:4,:],axis=0)

print('>%s: Average G-mean:%.3f(%.3f) ' % ('LR_mean:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('LR_mean:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('LR_mean:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('LR_mean:',np.mean(recall),np.std(recall)))    
print('>%s: Average accuracy_score: %.3f(%.3f)' % ('LR_mean:',np.mean(acc_s),np.std(acc_s)))

print('>%s: Average Training time: %.3f(%.3f)' % ('LR_mean:',np.mean(T),np.std(T)))
print('>%s: Average Score: %.3f(%.3f)' % ('LR_mean:',np.mean(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result)[-1,:]/10*9+1))))

# print('>%s: Average accuracy_score: %.3f(%.3f)' % ('IR='+str(k),np.mean(acc_s),np.std(acc_s)))

Optimal upper_mean selected by CV: 3.099 
              precision    recall  f1-score   support

         0.0       0.98      0.85      0.91       287
         1.0       0.12      0.60      0.20        10

    accuracy                           0.84       297
   macro avg       0.55      0.73      0.56       297
weighted avg       0.95      0.84      0.89       297

Optimal upper_mean selected by CV: 2.980 
              precision    recall  f1-score   support

         0.0       0.99      0.87      0.93       287
         1.0       0.17      0.80      0.29        10

    accuracy                           0.87       297
   macro avg       0.58      0.83      0.61       297
weighted avg       0.96      0.87      0.90       297

Optimal upper_mean selected by CV: 2.918 
              precision    recall  f1-score   support

         0.0       1.00      0.86      0.93       287
         1.0       0.20      1.00      0.34        10

    accuracy                           0.87       297
  

In [407]:
np.array(bias['upper_mean'])[0]

array([0.4696526 , 2.25753618])

# Volatility-uncertain method

In [76]:
def F_u(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[1]*st.norm.cdf(c/x[1])/(x[0]+x[1]))
        else:
            p.append(1-2*x[0]*st.norm.cdf(-c/x[0])/(x[0]+x[1]))
    return np.array(p)
def F_L(x,arr):
    p=[]
    for c in arr:
        
        if c <0:
            p.append(2*x[0]*st.norm.cdf(c/x[0])/(x[0]+x[1]))
        else:
            p.append(1-2*x[1]*st.norm.cdf(-c/x[1])/(x[0]+x[1]))
    return np.array(p)

In [83]:
def equa_vxh(x,n,r):
    pre_in=np.dot(X_train,w_2).reshape(x_tr.shape[0],)
    err_u=y_tr-F_u(x,pre_in)
    err_L=y_tr-F_L(x,pre_in)
    for k in range(y_tr.shape[0]):
    
        if y_tr[k]==1:
            err_u[k]=r*err_u[k]
            err_L[k]=r*err_L[k]
    
    return np.array([testjx.testjx.meanuncertainty(err_u,n)[1],testjx.testjx.meanuncertainty(err_L,n)[0]])

In [853]:
equa_vxh([0.1,0.3],100)

array([ 0.0663078, -0.1309536])

In [111]:
def meanuncertainty(x,n):
    r=[]
    for i in range(0,len(x)+1-n,n//5):
        r.append(np.mean(x[i:i+n]))
    return min(r),max(r)

In [158]:
def equa_v(x,n,r):
 
    pre_in=(np.dot(x_tr,model_in.coef_.T)+model_in.intercept_).reshape(x_tr.shape[0],)
    err_u=y_tr-F_u(x,pre_in)
    err_L=y_tr-F_L(x,pre_in)
    for k in range(y_tr.shape[0]):
    
        if y_tr[k]==1:
            err_u[k]=r*err_u[k]
            err_L[k]=r*err_L[k]
    
    return np.array([meanuncertainty(err_u,n)[1],meanuncertainty(err_L,n)[0]])

In [384]:
y_tr.shape[0]

552

In [54]:
def var_evaluate_model(X,y,m,model=LogisticRegression()):
    
    bacc,f2,rec,pre=list(),list(),list(),list()
   
#   
    for train_index,test_index in cv_.split(X,y):
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                model.fit(X[train_index],y[train_index])
                prob_y=F_u(m,(np.dot(X[test_index],model.coef_.T)+model.intercept_).reshape(X[test_index].shape[0],))
        
#                 auc.append(roc_auc_score(y[test_index],prob_y))
                
                prdict_y=np.round(prob_y)
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=2))
 
    return bacc,f2,rec,pre

In [153]:
def Fivetrails_var(X,y,model=LogisticRegression()):
    G_m,bacc,f2,rec,pre,T=list(),list(),list(),list(),list(),list()
    for train_index,test_index in cv.split(X,y):
                s_1=time.time()
#                 X_resampled_smote,y_resampled_smote=SMOTE().fit_resample(X[train_index], y[train_index])
                model.fit(X[train_index],y[train_index])
                t_1=time.time()
#                 mean_u=np.array([fsolve(lambda x:equa_fivem(x,n,0.5*(k+1),model,X[train_index],y[train_index]),0.5) for n in range(int(y[train_index].shape[0]*0.2),int(y[train_index].shape[0]*0.8),5)])
                all_results = []
                c=0
                # for k in mean_lr:
            #     strat=time.time()
                for i in var_lr:
                    result=np.mean(var_evaluate_model(X[train_index],y[train_index],i),axis=1)
                    metric_res = {'window':10+5*c,'upper_mean': i}
                    c+=1
                    for name, value in zip(metrics_names, result):
                #             print(name, ': ', value)
                            metric_res[name] = value


                    all_results.append(metric_res)
                eva=pd.DataFrame(all_results)
#                 eva.to_csv('{:}: datapre_diabetesCV_which_mu_to_set_inLRBIAS.csv'.format(k))
                bias=eva[eva.iloc[:,-4]==(eva.iloc[:,-4]).max()]
#                 end=time.time()
                print("Optimal volatility selected by CV: ",np.array(bias['upper_mean']))  
                prob_y=F_u(np.array(bias['upper_mean'])[0],(np.dot(X[test_index],model.coef_.T)+model.intercept_).reshape(X[test_index].shape[0],))
                
                
                prdict_y=np.round(prob_y)
                print(classification_report(y[test_index],prdict_y))
                rec.append(recall_score(y[test_index],prdict_y))
                pre.append(precision_score(y[test_index],prdict_y))
                G_m.append(np.sqrt(recall_score(y[test_index],prdict_y)*recall_score(y[test_index],prdict_y,pos_label=0)))
                bacc.append(balanced_accuracy_score(y[test_index],prdict_y))
                f2.append(fbeta_score(y[test_index],prdict_y,beta=max(2,np.log(y_sample.shape[0]/sum(y_sample)-1))))
                T.append(t_1-s_1)
    return G_m,bacc,f2,rec,pre,T


In [156]:
##yeast4
result_v = Fivetrails_var(X_sample,y_sample)
# 
G_mean=result_v[0]
Bacc=result_v[1]
# summarize performance
recall=result_v[3]
Fbeta=result_v[2]
# T=result[-1]
acc_s=np.mean(np.array(result_v)[:4,:],axis=0)
T=result_v[-1]
print('>%s: Average G-mean:%.3f(%.3f) ' % ('LR_volatility:',np.mean(G_mean),np.std(G_mean)))
print('>%s: Average Balanced_Acc: %.3f(%.3f) ' % ('LR_volatility:',np.mean(Bacc),np.std(Bacc)))
print('>%s: Average Fbeta: %.3f(%.3f)' % ('LR_volatility:',np.mean(Fbeta),np.std(Fbeta)))
print('>%s: Average Recall: %.3f(%.3f)' % ('LR_volatility:',np.mean(recall),np.std(recall)))    
print('>%s: Average Training time: %.3f(%.3f)' % ('LR_volatility:',np.mean(T),np.std(T)))
print('>%s: Average accuracy_score: %.3f(%.3f)' % ('LR_volatility:',np.mean(acc_s),np.std(acc_s)))
print('>%s: Average Score: %.3f(%.3f)' % ('LR_volatility:',np.mean(0.99*acc_s+0.01/(np.array(result_v)[-1,:]/10*9+1)),np.std(0.99*acc_s+0.01/(np.array(result_v)[-1,:]/10*9+1))))

Optimal volatility selected by CV:  [array([ 5.21935252, 10.08817084])]
              precision    recall  f1-score   support

         0.0       0.98      0.85      0.91       287
         1.0       0.12      0.60      0.20        10

    accuracy                           0.84       297
   macro avg       0.55      0.73      0.56       297
weighted avg       0.95      0.84      0.89       297

Optimal volatility selected by CV:  [array([4.94724769, 9.34293741])]
              precision    recall  f1-score   support

         0.0       0.98      0.89      0.94       287
         1.0       0.16      0.60      0.26        10

    accuracy                           0.88       297
   macro avg       0.57      0.75      0.60       297
weighted avg       0.96      0.88      0.91       297

Optimal volatility selected by CV:  [array([ 5.21935252, 10.08817084])]
              precision    recall  f1-score   support

         0.0       1.00      0.81      0.89       287
         1.0       0.15